In [41]:
%pwd

'C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops'

In [42]:
import os
from pathlib import Path
os.chdir(Path("C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops\\"))

In [43]:
%pwd

'C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops'

In [44]:
# from software_defect_prediction.constants import *
from software_defect_prediction.utils.common import *
# from software_defect_prediction.entity.config_entity import DataIngestionConfig
# from software_defect_prediction.config.configuration import ConfigurationManager

In [45]:
from dataclasses import dataclass
from pathlib import Path
from software_defect_prediction import logger

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    source_URL_file_name : str
    source_URL_unzip_file_name : str
    local_data_file: Path
    unzip_dir: Path

In [46]:
from software_defect_prediction.constants import *
from software_defect_prediction.utils.common import read_yaml, create_directories
from software_defect_prediction.entity.config_entity import (DataIngestionConfig)

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            source_URL_file_name = config.source_URL_file_name,
            source_URL_unzip_file_name = config.source_URL_unzip_file_name,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [47]:
configuration_obj = ConfigurationManager()

2024-05-24 09:33:37.501 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: config\config.yaml loaded successfully
2024-05-24 09:33:37.511 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: params.yaml loaded successfully
2024-05-24 09:33:37.519 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: schema.yaml loaded successfully
2024-05-24 09:33:37.522 | INFO     | software_defect_prediction.utils.common:create_directories:51 - created directory at: artifacts


In [48]:
data_ingestion_conf =  configuration_obj.get_data_ingestion_config()

2024-05-24 09:33:37.528 | INFO     | software_defect_prediction.utils.common:create_directories:51 - created directory at: artifacts/data_ingestion


In [49]:
data_ingestion_conf

DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://drive.usercontent.google.com/u/0/uc?id=1-16D_BjR9RGJJ-UySEqFKWSO7ElDmmVs&export=download', source_URL_file_name='software_defect_detection.zip', source_URL_unzip_file_name='train.csv', local_data_file='software_defect_detection.csv', unzip_dir='artifacts/data_ingestion')

In [50]:
%pwd

'C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops'

In [51]:
import urllib.request
from pathlib import Path

def download_file(url : str, save_path : str):
    try:
        # Open a connection to the URL
        with urllib.request.urlopen(url) as response:
            # Read data from the response
            data = response.read()
            
            # Write the data to a file at the specified save path
            with open(save_path, 'wb') as file:
                file.write(data)
                
        logger.info(f"File downloaded successfully to: {save_path}")
    except Exception as e:
        logger.error(f"Failed to download file from {url}: {e}")
        
url = data_ingestion_conf.source_URL  # URL of the file to download
save_path =  Path(data_ingestion_conf.root_dir,data_ingestion_conf.source_URL_file_name) # Path where the file will be saved

# Call the function to download the file
download_file(url, save_path)

2024-05-24 09:33:43.318 | INFO     | __main__:download_file:15 - File downloaded successfully to: artifacts\data_ingestion\software_defect_detection.zip


In [52]:
import zipfile

def unzip_file(zip_file_path, extract_dir, from_rename,to_rename):
    try:
        # Create the extraction directory if it doesn't exist
        os.makedirs(extract_dir, exist_ok=True)
        
        # Open the zip file for reading
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # Extract all contents of the zip file to the extraction directory
            zip_ref.extractall(extract_dir)
            os.replace(Path(extract_dir,from_rename),Path(extract_dir,to_rename))
            
        logger.info(f"Successfully extracted {zip_file_path} to {extract_dir}")
    except Exception as e:
        logger.error(f"Failed to unzip {zip_file_path}: {e}")

# Example usage
zip_file_path = Path(data_ingestion_conf.root_dir,data_ingestion_conf.source_URL_file_name)        # Path to the zip file to be extracted
extract_dir = data_ingestion_conf.unzip_dir   # Directory where contents will be extracted

# Call the function to unzip the file
unzip_file(zip_file_path, extract_dir, data_ingestion_conf.source_URL_unzip_file_name, data_ingestion_conf.local_data_file)

2024-05-24 09:33:43.425 | INFO     | __main__:unzip_file:14 - Successfully extracted artifacts\data_ingestion\software_defect_detection.zip to artifacts/data_ingestion
